In [39]:
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn import svm

from sklearn import tree
from sklearn import neighbors

from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF,DotProduct, WhiteKernel
from sklearn.gaussian_process.kernels import Matern
from sklearn.gaussian_process.kernels import RationalQuadratic


from sklearn.model_selection import cross_val_score, KFold

from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingRegressor

from sklearn import linear_model
from sklearn import ensemble 
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
from sklearn.neighbors import KNeighborsRegressor


In [40]:
cwd = os.getcwd()

# Read and Scale the Data

In [41]:
train_data = pd.read_excel (r'training_fixed.xlsx')

name=['Z','N','A','A^2/3','(N-Z)/A','PF','Zeo','NEO','Delta_Z','Delta_N','Shell_P','Shell_N']  

x_train = pd.DataFrame(train_data,  columns=name)  
y_train = pd.DataFrame(train_data, columns= ['MeV'])

test_size = 0.20  # You can adjust this to set the proportion of the test set
random_state = 100  # You can choose any random_state value for reproducibility

x_train, x_test, y_train, y_test = train_test_split(x_train, y_train, test_size=test_size, random_state=random_state)

train_len=len(x_train)
test_len=len(x_test)

#test_data= pd.read_excel (r'test_fixed.xls')
#x_test = pd.DataFrame(test_data,  columns=name)  
#y_test = pd.DataFrame(test_data, columns= ['MeV'])

#test_len=len(x_test)

extra_data= pd.read_excel (r'NEW_EXP_DATA_FOR_EXTRAPOLATION.xlsx')
x_extra = pd.DataFrame(extra_data,  columns=name)  
y_extra = pd.DataFrame(extra_data, columns= ['MeV'])

inp=np.concatenate((x_train,x_test,x_extra),axis=0)
out=np.concatenate((y_train,y_test,y_extra),axis=0)
train_test_len=train_len+test_len
##################################
## scale
##################################
#sc_X= StandardScaler()
#inp = sc_X.fit_transform(inp)
#sc_Y = StandardScaler()
#out=sc_Y.fit_transform(out)

#x_train=inp[0:train_len]
#x_test=inp[train_len:train_test_len]
#x_extra=inp[train_test_len::]
#y_train=out[0:train_len]
#y_test=out[train_len:train_test_len]
#y_extra=out[train_test_len::]
# Splitting the data

x_train = inp[0:train_len]
x_test = inp[train_len:train_test_len]
x_extra = inp[train_test_len::]
y_train = out[0:train_len]
y_test = out[train_len:train_test_len]
y_extra = out[train_test_len::]

##################################
## scale
##################################
# Initialize the StandardScaler for input data
sc_X = StandardScaler()

# Fit the scaler on the training data
sc_X.fit(x_train)

# Transform the training and testing data using the fitted scaler
x_train = sc_X.transform(x_train)
x_test = sc_X.transform(x_test)
x_extra = sc_X.transform(x_extra)

# Initialize the StandardScaler for output data
sc_Y = StandardScaler()

# Fit the scaler on the training data
sc_Y.fit(y_train)

# Transform the training and testing data using the fitted scaler
y_train = sc_Y.transform(y_train)
y_test = sc_Y.transform(y_test)
y_extra = sc_Y.transform(y_extra)

# For input data
print("Input Data:")
print("Mean:", sc_X.mean_)
print("Standard Deviation:", sc_X.scale_)

# For output data
print("\nOutput Data:")
print("Mean:", sc_Y.mean_)
print("Standard Deviation:", sc_Y.scale_)
################################
name+=['original','prediction']


Input Data:
Mean: [ 57.1975891   79.77777778 136.97536688  25.86887333   0.15163836
   3.02816209   0.471174     0.49161426   6.27410901   8.92138365
   1.63259958   2.24737945]
Standard Deviation: [24.55826201 37.92052664 62.07984918  8.4124406   0.07330345  2.39882469
  0.49916837  0.49992967  4.59671523  6.55112964  0.95590746  1.18455223]

Output Data:
Mean: [-40.42893003]
Standard Deviation: [42.04066295]


# GPR Model

### Kernel selected as RBF + White
#### length scale = (1e-2, 1e2) or (1e-5, 1e5) or ((1e-8, 1e8) 
#### noise level = (1e-10, 1e1) or (1e-10, 1e2) or  (1e-10, 1e-2)
#### alpha = 1e-05 or 1e-07 or 1e-09
#### nu = 0.5, 1.5, 2.5

In [ ]:
os.chdir(cwd)
analyze_name="GP_Regression"
if not (os.path.exists("./"+str(analyze_name))):
    os.makedirs("./"+str(analyze_name));
os.chdir("./"+str(analyze_name));
wfile=open("results_"+str(analyze_name)+"_selected.txt","a")
Scale=[1.0] 
Alpha=[1e-09] 
Normalize=[False] 

# K-fold cross-validation
#kf = KFold(n_splits=5, shuffle=True, random_state=42)

for s in Scale:
    for a in Alpha:
        for n in Normalize:
            wfile.write(analyze_name+"\n")
            wfile.write("Scale : {} Alpha: {} Normalize: {} \n".format(str(s),str(a),str(n)))
            wfile.write("----------------------------------\n")
            ###############################
            ## Model
            ################################
            #kernel = 1.0 * Matern(length_scale=1.0, nu=2.0,length_scale_bounds=(1e-10, 1e6))+\
            #WhiteKernel(noise_level=1, noise_level_bounds=(1e-20, 1e4))
            
            kernel = 1.0 * RBF(length_scale=s, length_scale_bounds=(1e-4, 1e5)) +\
            WhiteKernel(noise_level=1, noise_level_bounds=(1e-10, 1e1))
            gpr = GaussianProcessRegressor(kernel=kernel,alpha=a,normalize_y=n).fit(x_train, y_train)

        
            ######## EY: The score is printed  ########  31/10/2023 
            score = gpr.score(x_train, y_train)
             # Print the score
            print("R^2 Score:", score)
            ###########################################  

            # Cross-validation
            #cv_scores = cross_val_score(gpr, x_train, y_train.flatten(), cv=kf, scoring='neg_mean_squared_error')
            #mean_cv_score = np.mean(cv_scores)
            #wfile.write(f'Mean Cross-Validation Score: {mean_cv_score}\n')

            # Fit the GPR model on the entire training set
            #gpr.fit(x_train, y_train.flatten())
            
            
            pred_train=gpr.predict(x_train)  #EY: Train data added
            pred_train=pred_train.reshape(-1,1)  
                
            pred=gpr.predict(x_test)
            pred=pred.reshape(-1,1)
            
            pred_extra=gpr.predict(x_extra)
            pred_extra=pred_extra.reshape(-1,1)

            ########### EY: Standart deviations are calculated  ################### 31/10/2023
            pred_train_mean, pred_train_std = gpr.predict(x_train, return_std=True)     # Training set 
            pred_train_mean = pred_train_mean.reshape(-1, 1)  
            pred_train_std = pred_train_std.reshape(-1, 1)
            
            pred_mean, pred_std = gpr.predict(x_test, return_std=True)               # Test set
            pred_mean = pred_mean.reshape(-1, 1)
            pred_std = pred_std.reshape(-1, 1)
            
            pred_extra_mean, pred_extra_std = gpr.predict(x_extra, return_std=True)       # Extrapolation set 
            pred_extra_mean = pred_extra_mean.reshape(-1, 1)
            pred_extra_std = pred_extra_std.reshape(-1, 1)

            # Scale and normalize the standard deviations
            #sc_std = StandardScaler()
            #sc_std.fit(pred_train_std)  # Fit the scaler to the training standard deviations
            #pred_train_std_normalized = sc_std.transform(pred_train_std)
            #pred_std_normalized = sc_std.transform(pred_std)
            #pred_extra_std_normalized = sc_std.transform(pred_extra_std)
            #######################################################################  
            
            ################################
            ## Train data predictions
            ################################
            data=np.concatenate((sc_X.inverse_transform(x_train),sc_Y.inverse_transform(y_train),sc_Y.inverse_transform(pred_train)),axis=1)
            dfw = pd.DataFrame(data)
            dfw.to_excel('Train_data_predictions.xlsx', index=False, header=name)
    
            ################################
            ## Test data predictions
            ################################
            data=np.concatenate((sc_X.inverse_transform(x_test),sc_Y.inverse_transform(y_test),sc_Y.inverse_transform(pred)),axis=1)
            dfw = pd.DataFrame(data)
            dfw.to_excel('predictions.xlsx', index=False, header=name)
            ################################
            ## Extrapolation predictions
            ################################
            data=np.concatenate((sc_X.inverse_transform(x_extra),sc_Y.inverse_transform(y_extra),sc_Y.inverse_transform(pred_extra)),axis=1)
            dfw = pd.DataFrame(data)
            dfw.to_excel('extrapolation.xlsx', index=False, header=name)
            

            #################################  EY: The same as above but Standart deviations are also added.
            # Train Data Predictions (including standard deviations)
            data_train = np.concatenate((sc_X.inverse_transform(x_train), sc_Y.inverse_transform(y_train), sc_Y.inverse_transform(pred_train), sc_Y.inverse_transform(pred_train_std)-sc_Y.mean_), axis=1)
            dfw_train = pd.DataFrame(data_train)
            dfw_train.to_excel('Train_data_predictions_with_std.xlsx', index=False, header=name + ['Standard Deviation'])
            
            # Test Data Predictions (including standard deviations)
            data_test = np.concatenate((sc_X.inverse_transform(x_test), sc_Y.inverse_transform(y_test), sc_Y.inverse_transform(pred), sc_Y.inverse_transform(pred_std)-sc_Y.mean_), axis=1)
            dfw_test = pd.DataFrame(data_test)
            dfw_test.to_excel('predictions_with_std.xlsx', index=False, header=name + ['Standard Deviation'])
            
            # Extrapolation Predictions (including standard deviations)
            data_extra = np.concatenate((sc_X.inverse_transform(x_extra), sc_Y.inverse_transform(y_extra), sc_Y.inverse_transform(pred_extra), sc_Y.inverse_transform(pred_extra_std)-sc_Y.mean_), axis=1)
            dfw_extra = pd.DataFrame(data_extra)
            dfw_extra.to_excel('extrapolation_with_std.xlsx', index=False, header=name + ['Standard Deviation'])

            ################################
            ## Shap Part
            ################################            
            # Fits the explainer
            #explainer = shap.Explainer(gpr.predict, x_test)
            # Calculates the SHAP values - It takes some time
            #shap_values = explainer(x_test)
            
            #shap.summary_plot(shap_values,feature_names=name, plot_type='violin')


            #def gpr_predict(X):
             #   return gpr.predict(X).flatten()

            # Fits the explainer
            #explainer = shap.KernelExplainer(gpr_predict, shap.kmeans(x_train,4))
            # Calculates the SHAP values - It takes some time
            #shap_values = explainer.shap_values(x_test, nsamples='auto')
            
            #shap.summary_plot(shap_values, x_test, feature_names=name, plot_type='violin')
           # plt.show()
            # Create a new figure before saving to ensure the correct figure is saved
           # plt.figure()

            # Save the figure to a file (e.g., PNG)
           # plt.savefig('shap_summary_plot.pdf', dpi=600, bbox_inches='tight')
        
      
            ################################
            ## Write rms to txt file
            ################################
            mse_train = np.sum(np.square(sc_Y.inverse_transform(y_train) - sc_Y.inverse_transform(pred_train))) / np.size(y_train)
            rmse_train = np.sqrt(mse_train)
            
            mse = np.sum(np.square(sc_Y.inverse_transform(y_test) - sc_Y.inverse_transform(pred))) / np.size(y_test)
            rmse = np.sqrt(mse)
            
            ms_extra = np.sum(np.square(sc_Y.inverse_transform(y_extra) - sc_Y.inverse_transform(pred_extra))) / np.size(y_extra)
            rms_extra = np.sqrt(ms_extra)
            
            wfile.write("rms_train=%.4f,rms_pred=%.4f, rms_extra=%.4f\n"%(rmse_train, rmse, rms_extra))
            print(rmse_train, rmse,rms_extra)
                        
            wfile.close()
            os.chdir(cwd)
            

In [15]:
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, WhiteKernel, Matern, RationalQuadratic
from sklearn.metrics import mean_squared_error, r2_score
from joblib import Parallel, delayed

# Load the dataset
train_data = pd.read_excel('training_fixed.xlsx')

name = ['Z', 'N', 'A', 'A^2/3', '(N-Z)/A', 'PF', 'Zeo', 'NEO', 'Delta_Z', 'Delta_N', 'Shell_P', 'Shell_N']

x_train = pd.DataFrame(train_data, columns=name)
y_train = pd.DataFrame(train_data, columns=['MeV'])

test_size = 0.20  # Adjust this to set the proportion of the test set
random_state = 100  # Random state for reproducibility

x_train, x_test, y_train, y_test = train_test_split(x_train, y_train, test_size=test_size, random_state=random_state)

# Define parameter grids with specific bounds and fixed length_scale and noise_level
param_grid_rbf = {
    'kernel': [RBF(length_scale=1, length_scale_bounds=ls) + WhiteKernel(noise_level=1, noise_level_bounds=nl) for ls in [(1e-2, 1e2), (1e-5, 1e5), (1e-8, 1e8)] for nl in [(1e-10, 1e1), (1e-10, 1e2), (1e-10, 1e-2)]],
    'alpha': [1e-5, 1e-7, 1e-9]
}

param_grid_matern = {
    'kernel': [Matern(length_scale=1, length_scale_bounds=ls, nu=nu) + WhiteKernel(noise_level=1, noise_level_bounds=nl) for ls in [(1e-2, 1e2), (1e-5, 1e5), (1e-8, 1e8)] for nl in [(1e-10, 1e1), (1e-10, 1e2), (1e-10, 1e-2)] for nu in [0.5, 1.5, 2.5]],
    'alpha': [1e-5, 1e-7, 1e-9]
}

param_grid_rational_quadratic = {
    'kernel': [RationalQuadratic(length_scale=1, length_scale_bounds=ls) + WhiteKernel(noise_level=1, noise_level_bounds=nl) for ls in [(1e-2, 1e2), (1e-5, 1e5), (1e-8, 1e8)] for nl in [(1e-10, 1e1), (1e-10, 1e2), (1e-10, 1e-2)]],
    'alpha': [1e-5, 1e-7, 1e-9]
}

# Function to find the best parameters using GridSearchCV
def find_best_params(param_grid, x_train, y_train):
    gpr = GaussianProcessRegressor()
    grid_search = GridSearchCV(estimator=gpr, param_grid=param_grid, cv=5, n_jobs=-1, scoring='neg_mean_squared_error')
    grid_search.fit(x_train, y_train)
    return grid_search.best_params_, grid_search.best_estimator_

# Find the best parameters for each kernel function
best_params_rbf, best_model_rbf = find_best_params(param_grid_rbf, x_train, y_train)
best_params_matern, best_model_matern = find_best_params(param_grid_matern, x_train, y_train)
best_params_rational_quadratic, best_model_rational_quadratic = find_best_params(param_grid_rational_quadratic, x_train, y_train)

# Function to evaluate a single model
def evaluate_single_model(model, x_train, y_train, x_test, y_test):
    y_train_pred = model.predict(x_train)
    y_test_pred = model.predict(x_test)
    
    r2_train = r2_score(y_train, y_train_pred)
    r2_test = r2_score(y_test, y_test_pred)
    
    rmse_train = np.sqrt(mean_squared_error(y_train, y_train_pred))
    rmse_test = np.sqrt(mean_squared_error(y_test, y_test_pred))
    
    return r2_train, r2_test, rmse_train, rmse_test

# Evaluate all models in parallel
results = Parallel(n_jobs=-1)(
    delayed(evaluate_single_model)(model, x_train, y_train, x_test, y_test)
    for model in [best_model_rbf, best_model_matern, best_model_rational_quadratic]
)

(r2_train_rbf, r2_test_rbf, rmse_train_rbf, rmse_test_rbf), \
(r2_train_matern, r2_test_matern, rmse_train_matern, rmse_test_matern), \
(r2_train_rational_quadratic, r2_test_rational_quadratic, rmse_train_rational_quadratic, rmse_test_rational_quadratic) = results

# Print the results
print("RBF + White")
print("Best parameters:", best_model_rbf.kernel_)
print("R^2 train:", r2_train_rbf)
print("R^2 test:", r2_test_rbf)
print("Train RMSE:", rmse_train_rbf)
print("Test RMSE:", rmse_test_rbf)

print("\nMatérn + White")
print("Best parameters:", best_model_matern.kernel_)
print("R^2 train:", r2_train_matern)
print("R^2 test:", r2_test_matern)
print("Train RMSE:", rmse_train_matern)
print("Test RMSE:", rmse_test_matern)

print("\nRational Quadratic + White")
print("Best parameters:", best_model_rational_quadratic.kernel_)
print("R^2 train:", r2_train_rational_quadratic)
print("R^2 test:", r2_test_rational_quadratic)
print("Train RMSE:", rmse_train_rational_quadratic)
print("Test RMSE:", rmse_test_rational_quadratic)

/Users/Charlene/anaconda3/lib/python3.11/site-packages/sklearn/gaussian_process/kernels.py:452: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__noise_level is close to the specified upper bound 10.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
/Users/Charlene/anaconda3/lib/python3.11/site-packages/sklearn/gaussian_process/kernels.py:452: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__noise_level is close to the specified upper bound 10.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
/Users/Charlene/anaconda3/lib/python3.11/site-packages/sklearn/gaussian_process/kernels.py:452: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__noise_level is close to the specified upper bound 10.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
/Users/Charlene/anaconda3/lib/python3.11/site-packages/skl

/Users/Charlene/anaconda3/lib/python3.11/site-packages/sklearn/gaussian_process/kernels.py:452: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__length_scale is close to the specified upper bound 100000000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
/Users/Charlene/anaconda3/lib/python3.11/site-packages/sklearn/gaussian_process/kernels.py:452: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__noise_level is close to the specified upper bound 10.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
/Users/Charlene/anaconda3/lib/python3.11/site-packages/sklearn/gaussian_process/kernels.py:452: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__length_scale is close to the specified upper bound 100000000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
/Users/Charlene/anaconda3/lib/python3.11/s

/Users/Charlene/anaconda3/lib/python3.11/site-packages/sklearn/gaussian_process/kernels.py:452: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__noise_level is close to the specified upper bound 0.01. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
/Users/Charlene/anaconda3/lib/python3.11/site-packages/sklearn/gaussian_process/kernels.py:452: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__noise_level is close to the specified upper bound 0.01. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
/Users/Charlene/anaconda3/lib/python3.11/site-packages/sklearn/gaussian_process/kernels.py:452: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__noise_level is close to the specified upper bound 10.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
/Users/Charlene/anaconda3/lib/python3.11/site-packages/skl

/Users/Charlene/anaconda3/lib/python3.11/site-packages/sklearn/gaussian_process/kernels.py:452: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__length_scale is close to the specified upper bound 100000000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
/Users/Charlene/anaconda3/lib/python3.11/site-packages/sklearn/gaussian_process/kernels.py:452: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__noise_level is close to the specified upper bound 10.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
/Users/Charlene/anaconda3/lib/python3.11/site-packages/sklearn/gaussian_process/kernels.py:452: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__length_scale is close to the specified upper bound 100000000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
/Users/Charlene/anaconda3/lib/python3.11/s

/Users/Charlene/anaconda3/lib/python3.11/site-packages/sklearn/gaussian_process/kernels.py:452: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__noise_level is close to the specified upper bound 10.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
/Users/Charlene/anaconda3/lib/python3.11/site-packages/sklearn/gaussian_process/kernels.py:452: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__noise_level is close to the specified upper bound 10.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
/Users/Charlene/anaconda3/lib/python3.11/site-packages/sklearn/gaussian_process/kernels.py:452: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__noise_level is close to the specified upper bound 10.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
/Users/Charlene/anaconda3/lib/python3.11/site-packages/skl

...

/Users/Charlene/anaconda3/lib/python3.11/site-packages/sklearn/gaussian_process/kernels.py:442: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__alpha is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(
/Users/Charlene/anaconda3/lib/python3.11/site-packages/sklearn/gaussian_process/kernels.py:452: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__length_scale is close to the specified upper bound 100000000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
/Users/Charlene/anaconda3/lib/python3.11/site-packages/sklearn/gaussian_process/kernels.py:452: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__noise_level is close to the specified upper bound 100.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
/Users/Charlene/anaconda3/lib/python3.11/site-packages

/Users/Charlene/anaconda3/lib/python3.11/site-packages/sklearn/gaussian_process/kernels.py:452: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__alpha is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
/Users/Charlene/anaconda3/lib/python3.11/site-packages/sklearn/gaussian_process/kernels.py:452: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__noise_level is close to the specified upper bound 100.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
/Users/Charlene/anaconda3/lib/python3.11/site-packages/sklearn/gaussian_process/kernels.py:452: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__noise_level is close to the specified upper bound 0.01. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
/Users/Charlene/anaconda3/lib/python3.11/site-packages/skle

/Users/Charlene/anaconda3/lib/python3.11/site-packages/sklearn/gaussian_process/kernels.py:442: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__alpha is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(
/Users/Charlene/anaconda3/lib/python3.11/site-packages/sklearn/gaussian_process/kernels.py:452: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__length_scale is close to the specified upper bound 100000000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
/Users/Charlene/anaconda3/lib/python3.11/site-packages/sklearn/gaussian_process/kernels.py:452: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__noise_level is close to the specified upper bound 10.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
/Users/Charlene/anaconda3/lib/python3.11/site-packages/

/Users/Charlene/anaconda3/lib/python3.11/site-packages/sklearn/gaussian_process/kernels.py:452: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__noise_level is close to the specified upper bound 0.01. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
/Users/Charlene/anaconda3/lib/python3.11/site-packages/sklearn/gaussian_process/kernels.py:452: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__noise_level is close to the specified upper bound 0.01. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
/Users/Charlene/anaconda3/lib/python3.11/site-packages/sklearn/gaussian_process/kernels.py:452: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__alpha is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
/Users/Charlene/anaconda3/lib/python3.11/site-packages/sklea

RBF + White
Best parameters: RBF(length_scale=8.61) + WhiteKernel(noise_level=0.01)
R^2 train: 0.9999299009215495
R^2 test: 0.999799928956074
Train RMSE: 0.3519862594380718
Test RMSE: 0.5824669736031319

Matérn + White
Best parameters: Matern(length_scale=9.98, nu=2.5) + WhiteKernel(noise_level=0.01)
R^2 train: 0.9999701804859058
R^2 test: 0.9998239049404738
Train RMSE: 0.2295724859531751
Test RMSE: 0.5464529467772931

Rational Quadratic + White
Best parameters: RationalQuadratic(alpha=0.438, length_scale=4.69) + WhiteKernel(noise_level=0.01)
R^2 train: 0.9999924658470016
R^2 test: 0.9995515770757305
Train RMSE: 0.11539494208186118
Test RMSE: 0.872013261423959


In [16]:
# Print the results
print("RBF + White")
print("Best parameters:", best_model_rbf.kernel_)
print("Best alpha:", best_params_rbf['alpha'])
print("R^2 train:", r2_train_rbf)
print("R^2 test:", r2_test_rbf)
print("Train RMSE:", rmse_train_rbf)
print("Test RMSE:", rmse_test_rbf)

print("\nMatérn + White")
print("Best parameters:", best_model_matern.kernel_)
print("Best alpha:", best_params_matern['alpha'])
print("R^2 train:", r2_train_matern)
print("R^2 test:", r2_test_matern)
print("Train RMSE:", rmse_train_matern)
print("Test RMSE:", rmse_test_matern)

print("\nRational Quadratic + White")
print("Best parameters:", best_model_rational_quadratic.kernel_)
print("Best alpha:", best_params_rational_quadratic['alpha'])
print("R^2 train:", r2_train_rational_quadratic)
print("R^2 test:", r2_test_rational_quadratic)
print("Train RMSE:", rmse_train_rational_quadratic)
print("Test RMSE:", rmse_test_rational_quadratic)

RBF + White
Best parameters: RBF(length_scale=8.61) + WhiteKernel(noise_level=0.01)
Best alpha: 1e-09
R^2 train: 0.9999299009215495
R^2 test: 0.999799928956074
Train RMSE: 0.3519862594380718
Test RMSE: 0.5824669736031319

Matérn + White
Best parameters: Matern(length_scale=9.98, nu=2.5) + WhiteKernel(noise_level=0.01)
Best alpha: 1e-09
R^2 train: 0.9999701804859058
R^2 test: 0.9998239049404738
Train RMSE: 0.2295724859531751
Test RMSE: 0.5464529467772931

Rational Quadratic + White
Best parameters: RationalQuadratic(alpha=0.438, length_scale=4.69) + WhiteKernel(noise_level=0.01)
Best alpha: 1e-05
R^2 train: 0.9999924658470016
R^2 test: 0.9995515770757305
Train RMSE: 0.11539494208186118
Test RMSE: 0.872013261423959


In [18]:
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, WhiteKernel, Matern, RationalQuadratic
from sklearn.metrics import mean_squared_error, r2_score
from joblib import Parallel, delayed

# Load the dataset
train_data = pd.read_excel('training_fixed.xlsx')

name = ['Z', 'N', 'A', 'A^2/3', '(N-Z)/A', 'PF', 'Zeo', 'NEO', 'Delta_Z', 'Delta_N', 'Shell_P', 'Shell_N']

x_train = pd.DataFrame(train_data, columns=name)
y_train = pd.DataFrame(train_data, columns=['MeV'])

test_size = 0.30  # Adjust this to set the proportion of the test set
random_state = 100  # Random state for reproducibility

x_train, x_test, y_train, y_test = train_test_split(x_train, y_train, test_size=test_size, random_state=random_state)

# Define parameter grids with specific bounds and fixed length_scale and noise_level
param_grid_rbf = {
    'kernel': [RBF(length_scale=1, length_scale_bounds=ls) + WhiteKernel(noise_level=1, noise_level_bounds=nl) for ls in [(1e-2, 1e2), (1e-5, 1e5), (1e-8, 1e8)] for nl in [(1e-10, 1e1), (1e-10, 1e2), (1e-10, 1e-2)]],
    'alpha': [1e-5, 1e-7, 1e-9]
}

param_grid_matern = {
    'kernel': [Matern(length_scale=1, length_scale_bounds=ls, nu=nu) + WhiteKernel(noise_level=1, noise_level_bounds=nl) for ls in [(1e-2, 1e2), (1e-5, 1e5), (1e-8, 1e8)] for nl in [(1e-10, 1e1), (1e-10, 1e2), (1e-10, 1e-2)] for nu in [0.5, 1.5, 2.5]],
    'alpha': [1e-5, 1e-7, 1e-9]
}

param_grid_rational_quadratic = {
    'kernel': [RationalQuadratic(length_scale=1, length_scale_bounds=ls) + WhiteKernel(noise_level=1, noise_level_bounds=nl) for ls in [(1e-2, 1e2), (1e-5, 1e5), (1e-8, 1e8)] for nl in [(1e-10, 1e1), (1e-10, 1e2), (1e-10, 1e-2)]],
    'alpha': [1e-5, 1e-7, 1e-9]
}

# Function to find the best parameters using GridSearchCV
def find_best_params(param_grid, x_train, y_train):
    gpr = GaussianProcessRegressor()
    grid_search = GridSearchCV(estimator=gpr, param_grid=param_grid, cv=5, n_jobs=-1, scoring='neg_mean_squared_error')
    grid_search.fit(x_train, y_train)
    return grid_search.best_params_, grid_search.best_estimator_

# Find the best parameters for each kernel function
best_params_rbf, best_model_rbf = find_best_params(param_grid_rbf, x_train, y_train)
best_params_matern, best_model_matern = find_best_params(param_grid_matern, x_train, y_train)
best_params_rational_quadratic, best_model_rational_quadratic = find_best_params(param_grid_rational_quadratic, x_train, y_train)

# Function to evaluate a single model
def evaluate_single_model(model, x_train, y_train, x_test, y_test):
    y_train_pred = model.predict(x_train)
    y_test_pred = model.predict(x_test)
    
    r2_train = r2_score(y_train, y_train_pred)
    r2_test = r2_score(y_test, y_test_pred)
    
    rmse_train = np.sqrt(mean_squared_error(y_train, y_train_pred))
    rmse_test = np.sqrt(mean_squared_error(y_test, y_test_pred))
    
    return r2_train, r2_test, rmse_train, rmse_test

# Evaluate all models in parallel
results = Parallel(n_jobs=-1)(
    delayed(evaluate_single_model)(model, x_train, y_train, x_test, y_test)
    for model in [best_model_rbf, best_model_matern, best_model_rational_quadratic]
)

(r2_train_rbf, r2_test_rbf, rmse_train_rbf, rmse_test_rbf), \
(r2_train_matern, r2_test_matern, rmse_train_matern, rmse_test_matern), \
(r2_train_rational_quadratic, r2_test_rational_quadratic, rmse_train_rational_quadratic, rmse_test_rational_quadratic) = results

# Print the results
print("RBF + White")
print("Best parameters:", best_model_rbf.kernel_)
print("R^2 train:", r2_train_rbf)
print("R^2 test:", r2_test_rbf)
print("Train RMSE:", rmse_train_rbf)
print("Test RMSE:", rmse_test_rbf)

print("\nMatérn + White")
print("Best parameters:", best_model_matern.kernel_)
print("R^2 train:", r2_train_matern)
print("R^2 test:", r2_test_matern)
print("Train RMSE:", rmse_train_matern)
print("Test RMSE:", rmse_test_matern)

print("\nRational Quadratic + White")
print("Best parameters:", best_model_rational_quadratic.kernel_)
print("R^2 train:", r2_train_rational_quadratic)
print("R^2 test:", r2_test_rational_quadratic)
print("Train RMSE:", rmse_train_rational_quadratic)
print("Test RMSE:", rmse_test_rational_quadratic)

/Users/Charlene/anaconda3/lib/python3.11/site-packages/sklearn/gaussian_process/kernels.py:452: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__noise_level is close to the specified upper bound 10.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
/Users/Charlene/anaconda3/lib/python3.11/site-packages/sklearn/gaussian_process/kernels.py:452: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__noise_level is close to the specified upper bound 10.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
/Users/Charlene/anaconda3/lib/python3.11/site-packages/sklearn/gaussian_process/kernels.py:452: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__noise_level is close to the specified upper bound 10.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
/Users/Charlene/anaconda3/lib/python3.11/site-packages/skl

/Users/Charlene/anaconda3/lib/python3.11/site-packages/sklearn/gaussian_process/kernels.py:452: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__length_scale is close to the specified upper bound 100000000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
/Users/Charlene/anaconda3/lib/python3.11/site-packages/sklearn/gaussian_process/kernels.py:452: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__noise_level is close to the specified upper bound 10.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
/Users/Charlene/anaconda3/lib/python3.11/site-packages/sklearn/gaussian_process/kernels.py:452: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__noise_level is close to the specified upper bound 0.01. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
/Users/Charlene/anaconda3/lib/python3.11/site-pack

/Users/Charlene/anaconda3/lib/python3.11/site-packages/sklearn/gaussian_process/kernels.py:452: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__noise_level is close to the specified upper bound 10.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
/Users/Charlene/anaconda3/lib/python3.11/site-packages/sklearn/gaussian_process/kernels.py:452: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__noise_level is close to the specified upper bound 10.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
/Users/Charlene/anaconda3/lib/python3.11/site-packages/sklearn/gaussian_process/kernels.py:452: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__noise_level is close to the specified upper bound 10.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
/Users/Charlene/anaconda3/lib/python3.11/site-packages/skl

/Users/Charlene/anaconda3/lib/python3.11/site-packages/sklearn/gaussian_process/kernels.py:452: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__noise_level is close to the specified upper bound 0.01. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
/Users/Charlene/anaconda3/lib/python3.11/site-packages/sklearn/gaussian_process/kernels.py:452: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__length_scale is close to the specified upper bound 100000000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
/Users/Charlene/anaconda3/lib/python3.11/site-packages/sklearn/gaussian_process/kernels.py:452: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__noise_level is close to the specified upper bound 10.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
/Users/Charlene/anaconda3/lib/python3.11/site-pack

/Users/Charlene/anaconda3/lib/python3.11/site-packages/sklearn/gaussian_process/kernels.py:452: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__noise_level is close to the specified upper bound 10.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
/Users/Charlene/anaconda3/lib/python3.11/site-packages/sklearn/gaussian_process/kernels.py:452: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__noise_level is close to the specified upper bound 0.01. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
/Users/Charlene/anaconda3/lib/python3.11/site-packages/sklearn/gaussian_process/kernels.py:452: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__noise_level is close to the specified upper bound 10.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
/Users/Charlene/anaconda3/lib/python3.11/site-packages/skl

...

/Users/Charlene/anaconda3/lib/python3.11/site-packages/sklearn/gaussian_process/kernels.py:452: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__noise_level is close to the specified upper bound 0.01. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
/Users/Charlene/anaconda3/lib/python3.11/site-packages/sklearn/gaussian_process/kernels.py:442: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__alpha is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(
/Users/Charlene/anaconda3/lib/python3.11/site-packages/sklearn/gaussian_process/kernels.py:452: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__length_scale is close to the specified upper bound 100000000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
/Users/Charlene/anaconda3/lib/python3.11/site-packages/

/Users/Charlene/anaconda3/lib/python3.11/site-packages/sklearn/gaussian_process/kernels.py:452: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__noise_level is close to the specified upper bound 0.01. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
/Users/Charlene/anaconda3/lib/python3.11/site-packages/sklearn/gaussian_process/kernels.py:452: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__noise_level is close to the specified upper bound 0.01. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
/Users/Charlene/anaconda3/lib/python3.11/site-packages/sklearn/gaussian_process/kernels.py:452: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__noise_level is close to the specified upper bound 0.01. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
/Users/Charlene/anaconda3/lib/python3.11/site-packages/skl

/Users/Charlene/anaconda3/lib/python3.11/site-packages/sklearn/gaussian_process/kernels.py:442: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__alpha is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(
/Users/Charlene/anaconda3/lib/python3.11/site-packages/sklearn/gaussian_process/kernels.py:452: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__length_scale is close to the specified upper bound 100000000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
/Users/Charlene/anaconda3/lib/python3.11/site-packages/sklearn/gaussian_process/kernels.py:452: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__noise_level is close to the specified upper bound 10.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
/Users/Charlene/anaconda3/lib/python3.11/site-packages/

/Users/Charlene/anaconda3/lib/python3.11/site-packages/sklearn/gaussian_process/kernels.py:452: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__noise_level is close to the specified upper bound 0.01. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
/Users/Charlene/anaconda3/lib/python3.11/site-packages/sklearn/gaussian_process/kernels.py:452: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__noise_level is close to the specified upper bound 0.01. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
/Users/Charlene/anaconda3/lib/python3.11/site-packages/sklearn/gaussian_process/kernels.py:452: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__noise_level is close to the specified upper bound 0.01. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
/Users/Charlene/anaconda3/lib/python3.11/site-packages/skl

RBF + White
Best parameters: RBF(length_scale=8.68) + WhiteKernel(noise_level=0.01)
R^2 train: 0.9999266007270007
R^2 test: 0.9997593032280929
Train RMSE: 0.3609542864458676
Test RMSE: 0.6397014792066358

Matérn + White
Best parameters: Matern(length_scale=10.1, nu=2.5) + WhiteKernel(noise_level=0.01)
R^2 train: 0.9999680966090989
R^2 test: 0.9997750430073878
Train RMSE: 0.2379713897939555
Test RMSE: 0.6184320245522853

Rational Quadratic + White
Best parameters: RationalQuadratic(alpha=0.436, length_scale=4.71) + WhiteKernel(noise_level=0.01)
R^2 train: 0.9999918422297578
R^2 test: 0.9994532305844364
Train RMSE: 0.12033502683768801
Test RMSE: 0.96414938565538


In [19]:
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, WhiteKernel, Matern, RationalQuadratic
from sklearn.metrics import mean_squared_error, r2_score
from joblib import Parallel, delayed

# Load the dataset
train_data = pd.read_excel('training_fixed.xlsx')

name = ['Z', 'N', 'A', 'A^2/3', '(N-Z)/A', 'PF', 'Zeo', 'NEO', 'Delta_Z', 'Delta_N', 'Shell_P', 'Shell_N']

x_train = pd.DataFrame(train_data, columns=name)
y_train = pd.DataFrame(train_data, columns=['MeV'])

test_size = 0.40  # Adjust this to set the proportion of the test set
random_state = 100  # Random state for reproducibility

x_train, x_test, y_train, y_test = train_test_split(x_train, y_train, test_size=test_size, random_state=random_state)

# Define parameter grids with specific bounds and fixed length_scale and noise_level
param_grid_rbf = {
    'kernel': [RBF(length_scale=1, length_scale_bounds=ls) + WhiteKernel(noise_level=1, noise_level_bounds=nl) for ls in [(1e-2, 1e2), (1e-5, 1e5), (1e-8, 1e8)] for nl in [(1e-10, 1e1), (1e-10, 1e2), (1e-10, 1e-2)]],
    'alpha': [1e-5, 1e-7, 1e-9]
}

param_grid_matern = {
    'kernel': [Matern(length_scale=1, length_scale_bounds=ls, nu=nu) + WhiteKernel(noise_level=1, noise_level_bounds=nl) for ls in [(1e-2, 1e2), (1e-5, 1e5), (1e-8, 1e8)] for nl in [(1e-10, 1e1), (1e-10, 1e2), (1e-10, 1e-2)] for nu in [0.5, 1.5, 2.5]],
    'alpha': [1e-5, 1e-7, 1e-9]
}

param_grid_rational_quadratic = {
    'kernel': [RationalQuadratic(length_scale=1, length_scale_bounds=ls) + WhiteKernel(noise_level=1, noise_level_bounds=nl) for ls in [(1e-2, 1e2), (1e-5, 1e5), (1e-8, 1e8)] for nl in [(1e-10, 1e1), (1e-10, 1e2), (1e-10, 1e-2)]],
    'alpha': [1e-5, 1e-7, 1e-9]
}

# Function to find the best parameters using GridSearchCV
def find_best_params(param_grid, x_train, y_train):
    gpr = GaussianProcessRegressor()
    grid_search = GridSearchCV(estimator=gpr, param_grid=param_grid, cv=5, n_jobs=-1, scoring='neg_mean_squared_error')
    grid_search.fit(x_train, y_train)
    return grid_search.best_params_, grid_search.best_estimator_

# Find the best parameters for each kernel function
best_params_rbf, best_model_rbf = find_best_params(param_grid_rbf, x_train, y_train)
best_params_matern, best_model_matern = find_best_params(param_grid_matern, x_train, y_train)
best_params_rational_quadratic, best_model_rational_quadratic = find_best_params(param_grid_rational_quadratic, x_train, y_train)

# Function to evaluate a single model
def evaluate_single_model(model, x_train, y_train, x_test, y_test):
    y_train_pred = model.predict(x_train)
    y_test_pred = model.predict(x_test)
    
    r2_train = r2_score(y_train, y_train_pred)
    r2_test = r2_score(y_test, y_test_pred)
    
    rmse_train = np.sqrt(mean_squared_error(y_train, y_train_pred))
    rmse_test = np.sqrt(mean_squared_error(y_test, y_test_pred))
    
    return r2_train, r2_test, rmse_train, rmse_test

# Evaluate all models in parallel
results = Parallel(n_jobs=-1)(
    delayed(evaluate_single_model)(model, x_train, y_train, x_test, y_test)
    for model in [best_model_rbf, best_model_matern, best_model_rational_quadratic]
)

(r2_train_rbf, r2_test_rbf, rmse_train_rbf, rmse_test_rbf), \
(r2_train_matern, r2_test_matern, rmse_train_matern, rmse_test_matern), \
(r2_train_rational_quadratic, r2_test_rational_quadratic, rmse_train_rational_quadratic, rmse_test_rational_quadratic) = results

# Print the results
print("RBF + White")
print("Best parameters:", best_model_rbf.kernel_)
print("R^2 train:", r2_train_rbf)
print("R^2 test:", r2_test_rbf)
print("Train RMSE:", rmse_train_rbf)
print("Test RMSE:", rmse_test_rbf)

print("\nMatérn + White")
print("Best parameters:", best_model_matern.kernel_)
print("R^2 train:", r2_train_matern)
print("R^2 test:", r2_test_matern)
print("Train RMSE:", rmse_train_matern)
print("Test RMSE:", rmse_test_matern)

print("\nRational Quadratic + White")
print("Best parameters:", best_model_rational_quadratic.kernel_)
print("R^2 train:", r2_train_rational_quadratic)
print("R^2 test:", r2_test_rational_quadratic)
print("Train RMSE:", rmse_train_rational_quadratic)
print("Test RMSE:", rmse_test_rational_quadratic)

/Users/Charlene/anaconda3/lib/python3.11/site-packages/sklearn/gaussian_process/kernels.py:452: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__noise_level is close to the specified upper bound 10.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
/Users/Charlene/anaconda3/lib/python3.11/site-packages/sklearn/gaussian_process/kernels.py:452: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__noise_level is close to the specified upper bound 100.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
/Users/Charlene/anaconda3/lib/python3.11/site-packages/sklearn/gaussian_process/kernels.py:452: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__noise_level is close to the specified upper bound 10.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
/Users/Charlene/anaconda3/lib/python3.11/site-packages/sk

/Users/Charlene/anaconda3/lib/python3.11/site-packages/sklearn/gaussian_process/kernels.py:452: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__noise_level is close to the specified upper bound 0.01. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
/Users/Charlene/anaconda3/lib/python3.11/site-packages/sklearn/gaussian_process/kernels.py:452: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__length_scale is close to the specified upper bound 100000000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
/Users/Charlene/anaconda3/lib/python3.11/site-packages/sklearn/gaussian_process/kernels.py:452: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__noise_level is close to the specified upper bound 10.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
/Users/Charlene/anaconda3/lib/python3.11/site-pack

/Users/Charlene/anaconda3/lib/python3.11/site-packages/sklearn/gaussian_process/kernels.py:452: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__noise_level is close to the specified upper bound 100.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
/Users/Charlene/anaconda3/lib/python3.11/site-packages/sklearn/gaussian_process/kernels.py:452: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__noise_level is close to the specified upper bound 100.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
/Users/Charlene/anaconda3/lib/python3.11/site-packages/sklearn/gaussian_process/kernels.py:452: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__noise_level is close to the specified upper bound 100.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
/Users/Charlene/anaconda3/lib/python3.11/site-packages/

/Users/Charlene/anaconda3/lib/python3.11/site-packages/sklearn/gaussian_process/kernels.py:452: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__noise_level is close to the specified upper bound 0.01. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
/Users/Charlene/anaconda3/lib/python3.11/site-packages/sklearn/gaussian_process/kernels.py:452: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__noise_level is close to the specified upper bound 0.01. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
/Users/Charlene/anaconda3/lib/python3.11/site-packages/sklearn/gaussian_process/kernels.py:452: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__length_scale is close to the specified upper bound 100000000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
/Users/Charlene/anaconda3/lib/python3.11/site-pack

/Users/Charlene/anaconda3/lib/python3.11/site-packages/sklearn/gaussian_process/kernels.py:452: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__noise_level is close to the specified upper bound 100.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
/Users/Charlene/anaconda3/lib/python3.11/site-packages/sklearn/gaussian_process/kernels.py:452: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__noise_level is close to the specified upper bound 10.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
/Users/Charlene/anaconda3/lib/python3.11/site-packages/sklearn/gaussian_process/kernels.py:452: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__noise_level is close to the specified upper bound 100.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
/Users/Charlene/anaconda3/lib/python3.11/site-packages/s

...

/Users/Charlene/anaconda3/lib/python3.11/site-packages/sklearn/gaussian_process/kernels.py:442: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__alpha is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(
/Users/Charlene/anaconda3/lib/python3.11/site-packages/sklearn/gaussian_process/kernels.py:452: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__length_scale is close to the specified upper bound 100000000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
/Users/Charlene/anaconda3/lib/python3.11/site-packages/sklearn/gaussian_process/kernels.py:452: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__noise_level is close to the specified upper bound 100.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
/Users/Charlene/anaconda3/lib/python3.11/site-packages

/Users/Charlene/anaconda3/lib/python3.11/site-packages/sklearn/gaussian_process/kernels.py:452: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__alpha is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
/Users/Charlene/anaconda3/lib/python3.11/site-packages/sklearn/gaussian_process/kernels.py:452: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__noise_level is close to the specified upper bound 100.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
/Users/Charlene/anaconda3/lib/python3.11/site-packages/sklearn/gaussian_process/kernels.py:452: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__alpha is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
/Users/Charlene/anaconda3/lib/python3.11/site-packages/sklear

/Users/Charlene/anaconda3/lib/python3.11/site-packages/sklearn/gaussian_process/kernels.py:442: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__alpha is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(
/Users/Charlene/anaconda3/lib/python3.11/site-packages/sklearn/gaussian_process/kernels.py:452: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__length_scale is close to the specified upper bound 100000000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
/Users/Charlene/anaconda3/lib/python3.11/site-packages/sklearn/gaussian_process/kernels.py:452: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__noise_level is close to the specified upper bound 10.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
/Users/Charlene/anaconda3/lib/python3.11/site-packages/

/Users/Charlene/anaconda3/lib/python3.11/site-packages/sklearn/gaussian_process/kernels.py:452: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__noise_level is close to the specified upper bound 0.01. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
/Users/Charlene/anaconda3/lib/python3.11/site-packages/sklearn/gaussian_process/kernels.py:452: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__alpha is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
/Users/Charlene/anaconda3/lib/python3.11/site-packages/sklearn/gaussian_process/kernels.py:452: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__noise_level is close to the specified upper bound 100.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
/Users/Charlene/anaconda3/lib/python3.11/site-packages/skle

RBF + White
Best parameters: RBF(length_scale=8.74) + WhiteKernel(noise_level=0.01)
R^2 train: 0.9999230165144936
R^2 test: 0.9997327971731473
Train RMSE: 0.37060262210961653
Test RMSE: 0.6752819033937625

Matérn + White
Best parameters: Matern(length_scale=10.1, nu=2.5) + WhiteKernel(noise_level=0.01)
R^2 train: 0.9999660423696979
R^2 test: 0.9997395041666931
Train RMSE: 0.2461378430590986
Test RMSE: 0.6667529988826082

Rational Quadratic + White
Best parameters: RationalQuadratic(alpha=0.436, length_scale=4.74) + WhiteKernel(noise_level=0.01)
R^2 train: 0.9999910884541277
R^2 test: 0.9993669626498729
Train RMSE: 0.1260916204567672
Test RMSE: 1.0393917241949198


In [20]:
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, WhiteKernel, Matern, RationalQuadratic
from sklearn.metrics import mean_squared_error, r2_score
from joblib import Parallel, delayed

# Load the dataset
train_data = pd.read_excel('training_fixed.xlsx')

name = ['Z', 'N', 'A', 'A^2/3', '(N-Z)/A', 'PF', 'Zeo', 'NEO', 'Delta_Z', 'Delta_N', 'Shell_P', 'Shell_N']

x_train = pd.DataFrame(train_data, columns=name)
y_train = pd.DataFrame(train_data, columns=['MeV'])

test_size = 0.50  # Adjust this to set the proportion of the test set
random_state = 100  # Random state for reproducibility

x_train, x_test, y_train, y_test = train_test_split(x_train, y_train, test_size=test_size, random_state=random_state)

# Define parameter grids with specific bounds and fixed length_scale and noise_level
param_grid_rbf = {
    'kernel': [RBF(length_scale=1, length_scale_bounds=ls) + WhiteKernel(noise_level=1, noise_level_bounds=nl) for ls in [(1e-2, 1e2), (1e-5, 1e5), (1e-8, 1e8)] for nl in [(1e-10, 1e1), (1e-10, 1e2), (1e-10, 1e-2)]],
    'alpha': [1e-5, 1e-7, 1e-9]
}

param_grid_matern = {
    'kernel': [Matern(length_scale=1, length_scale_bounds=ls, nu=nu) + WhiteKernel(noise_level=1, noise_level_bounds=nl) for ls in [(1e-2, 1e2), (1e-5, 1e5), (1e-8, 1e8)] for nl in [(1e-10, 1e1), (1e-10, 1e2), (1e-10, 1e-2)] for nu in [0.5, 1.5, 2.5]],
    'alpha': [1e-5, 1e-7, 1e-9]
}

param_grid_rational_quadratic = {
    'kernel': [RationalQuadratic(length_scale=1, length_scale_bounds=ls) + WhiteKernel(noise_level=1, noise_level_bounds=nl) for ls in [(1e-2, 1e2), (1e-5, 1e5), (1e-8, 1e8)] for nl in [(1e-10, 1e1), (1e-10, 1e2), (1e-10, 1e-2)]],
    'alpha': [1e-5, 1e-7, 1e-9]
}

# Function to find the best parameters using GridSearchCV
def find_best_params(param_grid, x_train, y_train):
    gpr = GaussianProcessRegressor()
    grid_search = GridSearchCV(estimator=gpr, param_grid=param_grid, cv=5, n_jobs=-1, scoring='neg_mean_squared_error')
    grid_search.fit(x_train, y_train)
    return grid_search.best_params_, grid_search.best_estimator_

# Find the best parameters for each kernel function
best_params_rbf, best_model_rbf = find_best_params(param_grid_rbf, x_train, y_train)
best_params_matern, best_model_matern = find_best_params(param_grid_matern, x_train, y_train)
best_params_rational_quadratic, best_model_rational_quadratic = find_best_params(param_grid_rational_quadratic, x_train, y_train)

# Function to evaluate a single model
def evaluate_single_model(model, x_train, y_train, x_test, y_test):
    y_train_pred = model.predict(x_train)
    y_test_pred = model.predict(x_test)
    
    r2_train = r2_score(y_train, y_train_pred)
    r2_test = r2_score(y_test, y_test_pred)
    
    rmse_train = np.sqrt(mean_squared_error(y_train, y_train_pred))
    rmse_test = np.sqrt(mean_squared_error(y_test, y_test_pred))
    
    return r2_train, r2_test, rmse_train, rmse_test

# Evaluate all models in parallel
results = Parallel(n_jobs=-1)(
    delayed(evaluate_single_model)(model, x_train, y_train, x_test, y_test)
    for model in [best_model_rbf, best_model_matern, best_model_rational_quadratic]
)

(r2_train_rbf, r2_test_rbf, rmse_train_rbf, rmse_test_rbf), \
(r2_train_matern, r2_test_matern, rmse_train_matern, rmse_test_matern), \
(r2_train_rational_quadratic, r2_test_rational_quadratic, rmse_train_rational_quadratic, rmse_test_rational_quadratic) = results

# Print the results
print("RBF + White")
print("Best parameters:", best_model_rbf.kernel_)
print("R^2 train:", r2_train_rbf)
print("R^2 test:", r2_test_rbf)
print("Train RMSE:", rmse_train_rbf)
print("Test RMSE:", rmse_test_rbf)

print("\nMatérn + White")
print("Best parameters:", best_model_matern.kernel_)
print("R^2 train:", r2_train_matern)
print("R^2 test:", r2_test_matern)
print("Train RMSE:", rmse_train_matern)
print("Test RMSE:", rmse_test_matern)

print("\nRational Quadratic + White")
print("Best parameters:", best_model_rational_quadratic.kernel_)
print("R^2 train:", r2_train_rational_quadratic)
print("R^2 test:", r2_test_rational_quadratic)
print("Train RMSE:", rmse_train_rational_quadratic)
print("Test RMSE:", rmse_test_rational_quadratic)

/Users/Charlene/anaconda3/lib/python3.11/site-packages/sklearn/gaussian_process/kernels.py:452: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__noise_level is close to the specified upper bound 10.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
/Users/Charlene/anaconda3/lib/python3.11/site-packages/sklearn/gaussian_process/kernels.py:452: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__noise_level is close to the specified upper bound 100.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
/Users/Charlene/anaconda3/lib/python3.11/site-packages/sklearn/gaussian_process/kernels.py:452: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__noise_level is close to the specified upper bound 10.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
/Users/Charlene/anaconda3/lib/python3.11/site-packages/sk

/Users/Charlene/anaconda3/lib/python3.11/site-packages/sklearn/gaussian_process/kernels.py:452: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__noise_level is close to the specified upper bound 0.01. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
/Users/Charlene/anaconda3/lib/python3.11/site-packages/sklearn/gaussian_process/kernels.py:452: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__noise_level is close to the specified upper bound 0.01. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
/Users/Charlene/anaconda3/lib/python3.11/site-packages/sklearn/gaussian_process/kernels.py:452: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__length_scale is close to the specified upper bound 100000000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
/Users/Charlene/anaconda3/lib/python3.11/site-pack

/Users/Charlene/anaconda3/lib/python3.11/site-packages/sklearn/gaussian_process/kernels.py:452: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__noise_level is close to the specified upper bound 0.01. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
/Users/Charlene/anaconda3/lib/python3.11/site-packages/sklearn/gaussian_process/kernels.py:452: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__noise_level is close to the specified upper bound 10.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
/Users/Charlene/anaconda3/lib/python3.11/site-packages/sklearn/gaussian_process/kernels.py:452: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__noise_level is close to the specified upper bound 0.01. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
/Users/Charlene/anaconda3/lib/python3.11/site-packages/skl

/Users/Charlene/anaconda3/lib/python3.11/site-packages/sklearn/gaussian_process/kernels.py:452: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__length_scale is close to the specified upper bound 100000000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
/Users/Charlene/anaconda3/lib/python3.11/site-packages/sklearn/gaussian_process/kernels.py:452: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__noise_level is close to the specified upper bound 10.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
/Users/Charlene/anaconda3/lib/python3.11/site-packages/sklearn/gaussian_process/kernels.py:452: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__length_scale is close to the specified upper bound 100000000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
/Users/Charlene/anaconda3/lib/python3.11/s

/Users/Charlene/anaconda3/lib/python3.11/site-packages/sklearn/gaussian_process/kernels.py:452: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__noise_level is close to the specified upper bound 0.01. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
/Users/Charlene/anaconda3/lib/python3.11/site-packages/sklearn/gaussian_process/kernels.py:452: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__noise_level is close to the specified upper bound 0.01. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
/Users/Charlene/anaconda3/lib/python3.11/site-packages/sklearn/gaussian_process/kernels.py:452: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__noise_level is close to the specified upper bound 10.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
/Users/Charlene/anaconda3/lib/python3.11/site-packages/skl

...

/Users/Charlene/anaconda3/lib/python3.11/site-packages/sklearn/gaussian_process/kernels.py:442: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__alpha is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(
/Users/Charlene/anaconda3/lib/python3.11/site-packages/sklearn/gaussian_process/kernels.py:452: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__length_scale is close to the specified upper bound 100000000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
/Users/Charlene/anaconda3/lib/python3.11/site-packages/sklearn/gaussian_process/kernels.py:452: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__noise_level is close to the specified upper bound 10.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
/Users/Charlene/anaconda3/lib/python3.11/site-packages/

/Users/Charlene/anaconda3/lib/python3.11/site-packages/sklearn/gaussian_process/kernels.py:452: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__noise_level is close to the specified upper bound 0.01. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
/Users/Charlene/anaconda3/lib/python3.11/site-packages/sklearn/gaussian_process/kernels.py:452: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__noise_level is close to the specified upper bound 0.01. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
/Users/Charlene/anaconda3/lib/python3.11/site-packages/sklearn/gaussian_process/kernels.py:452: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__alpha is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
/Users/Charlene/anaconda3/lib/python3.11/site-packages/sklea

/Users/Charlene/anaconda3/lib/python3.11/site-packages/sklearn/gaussian_process/kernels.py:452: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__noise_level is close to the specified upper bound 10.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
/Users/Charlene/anaconda3/lib/python3.11/site-packages/sklearn/gaussian_process/kernels.py:452: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__alpha is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
/Users/Charlene/anaconda3/lib/python3.11/site-packages/sklearn/gaussian_process/kernels.py:452: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__noise_level is close to the specified upper bound 100.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
/Users/Charlene/anaconda3/lib/python3.11/site-packages/skle

/Users/Charlene/anaconda3/lib/python3.11/site-packages/sklearn/gaussian_process/kernels.py:442: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__alpha is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(
/Users/Charlene/anaconda3/lib/python3.11/site-packages/sklearn/gaussian_process/kernels.py:452: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__length_scale is close to the specified upper bound 100000000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
/Users/Charlene/anaconda3/lib/python3.11/site-packages/sklearn/gaussian_process/kernels.py:452: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__noise_level is close to the specified upper bound 100.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
/Users/Charlene/anaconda3/lib/python3.11/site-packages

RBF + White
Best parameters: RBF(length_scale=8.72) + WhiteKernel(noise_level=0.01)
R^2 train: 0.9999180197763577
R^2 test: 0.9996566985768089
Train RMSE: 0.3842636322865417
Test RMSE: 0.7648525408355075

Matérn + White
Best parameters: Matern(length_scale=10.1, nu=2.5) + WhiteKernel(noise_level=0.01)
R^2 train: 0.9999632810396941
R^2 test: 0.999657048410586
Train RMSE: 0.2571697733570941
Test RMSE: 0.7644627384118424

Rational Quadratic + White
Best parameters: RationalQuadratic(alpha=0.432, length_scale=4.77) + WhiteKernel(noise_level=0.01)
R^2 train: 0.99999056800532
R^2 test: 0.9990299608057415
Train RMSE: 0.1303396372110059
Test RMSE: 1.285684928609043


In [38]:
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, WhiteKernel, Matern, RationalQuadratic
from sklearn.metrics import mean_squared_error, r2_score
from joblib import Parallel, delayed

# Load the dataset
train_data = pd.read_excel('training_fixed.xlsx')

name = ['Z', 'N', 'A', 'A^2/3', '(N-Z)/A', 'PF', 'Zeo', 'NEO', 'Delta_Z', 'Delta_N', 'Shell_P', 'Shell_N']

x_train = pd.DataFrame(train_data, columns=name)
y_train = pd.DataFrame(train_data, columns=['MeV'])

test_size = 0.60  # Adjust this to set the proportion of the test set
random_state = 100  # Random state for reproducibility

x_train, x_test, y_train, y_test = train_test_split(x_train, y_train, test_size=test_size, random_state=random_state)

# Define parameter grids with specific bounds and fixed length_scale and noise_level
param_grid_rbf = {
    'kernel': [RBF(length_scale=1, length_scale_bounds=ls) + WhiteKernel(noise_level=1, noise_level_bounds=nl) for ls in [(1e-2, 1e2), (1e-5, 1e5), (1e-8, 1e8)] for nl in [(1e-10, 1e1), (1e-10, 1e2), (1e-10, 1e-2)]],
    'alpha': [1e-5, 1e-7, 1e-9]
}

param_grid_matern = {
    'kernel': [Matern(length_scale=1, length_scale_bounds=ls, nu=nu) + WhiteKernel(noise_level=1, noise_level_bounds=nl) for ls in [(1e-2, 1e2), (1e-5, 1e5), (1e-8, 1e8)] for nl in [(1e-10, 1e1), (1e-10, 1e2), (1e-10, 1e-2)] for nu in [0.5, 1.5, 2.5]],
    'alpha': [1e-5, 1e-7, 1e-9]
}

param_grid_rational_quadratic = {
    'kernel': [RationalQuadratic(length_scale=1, length_scale_bounds=ls) + WhiteKernel(noise_level=1, noise_level_bounds=nl) for ls in [(1e-2, 1e2), (1e-5, 1e5), (1e-8, 1e8)] for nl in [(1e-10, 1e1), (1e-10, 1e2), (1e-10, 1e-2)]],
    'alpha': [1e-5, 1e-7, 1e-9]
}

# Function to find the best parameters using GridSearchCV
def find_best_params(param_grid, x_train, y_train):
    gpr = GaussianProcessRegressor()
    grid_search = GridSearchCV(estimator=gpr, param_grid=param_grid, cv=5, n_jobs=-1, scoring='neg_mean_squared_error')
    grid_search.fit(x_train, y_train)
    return grid_search.best_params_, grid_search.best_estimator_

# Find the best parameters for each kernel function
best_params_rbf, best_model_rbf = find_best_params(param_grid_rbf, x_train, y_train)
best_params_matern, best_model_matern = find_best_params(param_grid_matern, x_train, y_train)
best_params_rational_quadratic, best_model_rational_quadratic = find_best_params(param_grid_rational_quadratic, x_train, y_train)

# Function to evaluate a single model
def evaluate_single_model(model, x_train, y_train, x_test, y_test):
    y_train_pred = model.predict(x_train)
    y_test_pred = model.predict(x_test)
    
    r2_train = r2_score(y_train, y_train_pred)
    r2_test = r2_score(y_test, y_test_pred)
    
    rmse_train = np.sqrt(mean_squared_error(y_train, y_train_pred))
    rmse_test = np.sqrt(mean_squared_error(y_test, y_test_pred))
    
    return r2_train, r2_test, rmse_train, rmse_test

# Evaluate all models in parallel
results = Parallel(n_jobs=-1)(
    delayed(evaluate_single_model)(model, x_train, y_train, x_test, y_test)
    for model in [best_model_rbf, best_model_matern, best_model_rational_quadratic]
)

(r2_train_rbf, r2_test_rbf, rmse_train_rbf, rmse_test_rbf), \
(r2_train_matern, r2_test_matern, rmse_train_matern, rmse_test_matern), \
(r2_train_rational_quadratic, r2_test_rational_quadratic, rmse_train_rational_quadratic, rmse_test_rational_quadratic) = results

# Extract the best parameters including the bounds
best_length_scale_bounds_rbf = best_model_rbf.kernel_.k1.bounds['length_scale']
best_noise_level_bounds_rbf = best_model_rbf.kernel_.k2.bounds['noise_level']

best_length_scale_bounds_matern = best_model_matern.kernel_.k1.bounds['length_scale']
best_noise_level_bounds_matern = best_model_matern.kernel_.k2.bounds['noise_level']

best_length_scale_bounds_rational_quadratic = best_model_rational_quadratic.kernel_.k1.bounds['length_scale']
best_noise_level_bounds_rational_quadratic = best_model_rational_quadratic.kernel_.k2.bounds['noise_level']

# Print the results
print("RBF + White")
print("Best parameters:", best_model_rbf.kernel_)
print("Best alpha:", best_params_rbf['alpha'])
print("Best length_scale_bounds:", best_length_scale_bounds_rbf)
print("Best noise_level_bounds:", best_noise_level_bounds_rbf)
print("R^2 train:", r2_train_rbf)
print("R^2 test:", r2_test_rbf)
print("Train RMSE:", rmse_train_rbf)
print("Test RMSE:", rmse_test_rbf)

print("\nMatérn + White")
print("Best parameters:", best_model_matern.kernel_)
print("Best alpha:", best_params_matern['alpha'])
print("Best length_scale_bounds:", best_length_scale_bounds_matern)
print("Best noise_level_bounds:", best_noise_level_bounds_matern)
print("R^2 train:", r2_train_matern)
print("R^2 test:", r2_test_matern)
print("Train RMSE:", rmse_train_matern)
print("Test RMSE:", rmse_test_matern)

print("\nRational Quadratic + White")
print("Best parameters:", best_model_rational_quadratic.kernel_)
print("Best alpha:", best_params_rational_quadratic['alpha'])
print("Best length_scale_bounds:", best_length_scale_bounds_rational_quadratic)
print("Best noise_level_bounds:", best_noise_level_bounds_rational_quadratic)
print("R^2 train:", r2_train_rational_quadratic)
print("R^2 test:", r2_test_rational_quadratic)
print("Train RMSE:", rmse_train_rational_quadratic)
print("Test RMSE:", rmse_test_rational_quadratic)


KeyboardInterrupt: 